In [1]:
# Dependencies
import requests
import bs4
from datetime import datetime
from pytz import timezone
import pandas as pd
import numpy as np
from urllib.request import Request, urlopen

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [2]:
# Make a reference to the books.csv file path
csv_path = "Resources/180818_Amazon_ProductId.csv"

# Import the books.csv file as a DataFrame
amazon_productid_df = pd.read_csv(csv_path, encoding="utf-8")
amazon_productid_df

,Product,asin
0,XA2_Black,B077ZKFB2D
1,XA2_Silver,B077ZL594Y
2,XA2_Pink,B077ZLD4PP
3,XA2_Blue,B077ZG9DHC
4,XA2_Ultra_Black,B077ZRWXPF
5,XA2_Ultra_Silver,B077ZKKDWK
6,XA2_Ultra_Gold,B077ZRWXPH
7,XA2_Ultra_Blue,B077ZKKBDB
8,L2_Black,B077ZMWX1W
9,L2_Pink,B077ZKF4N8


In [3]:
amazon_productid_df["asin"] = amazon_productid_df["asin"].astype(object)

In [4]:
amazon_asin_list = amazon_productid_df["asin"]

In [5]:
master_df = None

In [6]:
for asin in amazon_asin_list:

    
    options = Options()

    options.set_headless(True)

    driver = webdriver.Chrome(executable_path='chromedriver.exe')

    driver.get(f"https://www.amazon.com/gp/offer-listing/{asin}/ref=dp_olp_all_mbc?_encoding=UTF8&condition=new")
    
    time.sleep(1)

    html = driver.page_source.encode('utf-8')

    soup = bs4.BeautifulSoup(html, "html.parser")
    
    offer_blocks = soup.select('.a-row.a-spacing-mini.olpOffer')
       
    # Making empty lists
    seller_list = []
    price_list = []
        
    # Making seller_list
    for offer in offer_blocks:
    
        offer_seller = offer.select('.a-spacing-none.olpSellerName')
        seller = offer_seller[0].getText().strip()
        if seller == "":
            seller = "Amazon.com"
        seller = seller + " in Amazon" 
    
        offer_price = offer.select('.a-size-large.a-color-price.olpOfferPrice.a-text-bold')
        if offer_price == []:
            price = ""
        else:
            price = offer_price[0].getText().strip()
            price = price.replace("$", "")
            price = price.replace(",", "")
            price = float(price)
        
        if seller in seller_list:
            continue
        
        else:
            seller_list.append(seller)
            price_list.append(price)
                
    product_price_df = pd.DataFrame({"Seller": seller_list,
                                         asin: price_list})

    product_price_df = product_price_df.set_index("Seller")

    master_df = pd.concat([master_df, product_price_df], axis=1, sort=False)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use setter for headless property instead of set_headless
  import sys


In [7]:
master_df = master_df.T
master_df = master_df.reset_index()
master_df = master_df.rename(columns={"index": "asin"})
master_df["asin"] = master_df["asin"].astype(object)
master_df = pd.merge(amazon_productid_df, master_df, on="asin", how="left")
master_df = master_df.rename(columns={"asin": "ASIN"})
master_df

,Product,ASIN,S&Y ELECTRONICS in Amazon,MLN online in Amazon,I-BE in Amazon,Tripod Tech Inc in Amazon,DEALS#1 in Amazon,FOREVER ITEMS in Amazon,Riz Electronics in Amazon,Focus Camera LLC in Amazon,...,PAY NO TAX in Amazon,GreenLight Global LLC in Amazon,TheFactoryDepot in Amazon,BREED in Amazon,Triplenet Pricing INC in Amazon,Interloper Inc in Amazon,DealsAllDay13 in Amazon,Printermalls in Amazon,Mobiles International LLC in Amazon,PopularElectronicsInc. in Amazon
0,XA2_Black,B077ZKFB2D,289.00,299.99,364.21,400.0,475.2,499.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XA2_Silver,B077ZL594Y,284.25,NaN,NaN,NaN,432.0,499.0,284.25,299.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,XA2_Pink,B077ZLD4PP,NaN,NaN,364.21,NaN,NaN,NaN,NaN,299.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,XA2_Blue,B077ZG9DHC,278.00,NaN,NaN,NaN,388.8,499.0,278.00,299.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,XA2_Ultra_Black,B077ZRWXPF,NaN,NaN,445.29,NaN,350.0,499.0,294.45,249.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,XA2_Ultra_Silver,B077ZKKDWK,NaN,NaN,437.88,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,XA2_Ultra_Gold,B077ZRWXPH,NaN,NaN,437.88,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,XA2_Ultra_Blue,B077ZKKBDB,NaN,NaN,437.88,NaN,NaN,NaN,NaN,NaN,...,319.0,370.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,L2_Black,B077ZMWX1W,NaN,NaN,NaN,NaN,288.0,NaN,NaN,199.99,...,NaN,NaN,195.98,199.98,212.93,254.37,NaN,NaN,NaN,NaN
9,L2_Pink,B077ZKF4N8,NaN,NaN,NaN,NaN,195.0,NaN,NaN,199.99,...,NaN,NaN,NaN,199.98,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
amazon_result_df = master_df.drop(columns=["ASIN"])
amazon_result_df

,Product,S&Y ELECTRONICS in Amazon,MLN online in Amazon,I-BE in Amazon,Tripod Tech Inc in Amazon,DEALS#1 in Amazon,FOREVER ITEMS in Amazon,Riz Electronics in Amazon,Focus Camera LLC in Amazon,ⒽomeⓄnline in Amazon,...,PAY NO TAX in Amazon,GreenLight Global LLC in Amazon,TheFactoryDepot in Amazon,BREED in Amazon,Triplenet Pricing INC in Amazon,Interloper Inc in Amazon,DealsAllDay13 in Amazon,Printermalls in Amazon,Mobiles International LLC in Amazon,PopularElectronicsInc. in Amazon
0,XA2_Black,289.00,299.99,364.21,400.0,475.2,499.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XA2_Silver,284.25,NaN,NaN,NaN,432.0,499.0,284.25,299.99,412.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,XA2_Pink,NaN,NaN,364.21,NaN,NaN,NaN,NaN,299.99,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,XA2_Blue,278.00,NaN,NaN,NaN,388.8,499.0,278.00,299.99,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,XA2_Ultra_Black,NaN,NaN,445.29,NaN,350.0,499.0,294.45,249.99,348.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,XA2_Ultra_Silver,NaN,NaN,437.88,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,XA2_Ultra_Gold,NaN,NaN,437.88,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,XA2_Ultra_Blue,NaN,NaN,437.88,NaN,NaN,NaN,NaN,NaN,NaN,...,319.0,370.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,L2_Black,NaN,NaN,NaN,NaN,288.0,NaN,NaN,199.99,277.29,...,NaN,NaN,195.98,199.98,212.93,254.37,NaN,NaN,NaN,NaN
9,L2_Pink,NaN,NaN,NaN,NaN,195.0,NaN,NaN,199.99,NaN,...,NaN,NaN,NaN,199.98,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
amazon_result_df.to_csv("Output/amazon_result.csv", index=False)